Dans ce billet, nous allons [comme il y a quelques semaines](http://flothesof.github.io/podcast-rendez-vous-avec-X-fr.html), construire un flux RSS pour mon émission radio préférée : *Sur les épaules de Darwin*, animée par l'extraordinaire Jean Claude Ameisen.

Afin de faire ceci, nous allons partir du [travail de Clément Grimal](http://clementgrimal.fr/darwin/), qui maintient une liste à jour des émissions avec des liens de téléchargement. Une question, avant de commencer. Pourquoi faire ce travail, alors que le site de Clément permet déjà d'écouter à volonté ces émissions ? Tout simplement parce que son site ne permet pas de faire une recherche plein texte sur le contenu des émissions et également parce que j'aime écouter les émissions dans un logiciel de podcast afin de marquer ma progression.

Le lien vers le flux RSS généré à partir du présent notebook est ici : **https://raw.githubusercontent.com/flothesof/posts/master/files/podcast_Sur_les_epaules_de_Darwin.xml**.

# Récupération des liens vers les émissions

In [1]:
base_url = "http://clementgrimal.fr/darwin/"

In [2]:
import requests

In [3]:
r = requests.get(base_url)
r.encoding = 'utf-8'

On utilise beautiful soup pour déconstruire le résultat et chercher les différents liens dont nous avons besoin :

- titre de l'émission
- date de diffusion
- lien de téléchargement
- lien vers la page du site France Inter (pour en extraire la description de l'émission)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(r.text, 'html.parser')

On peut extraire toutes les émissions comme ceci :

In [6]:
all_shows = ([item for item in (li.find('table') for li in soup.findAll('li')) if item])

In [7]:
len(all_shows)

385

A partir de la première émission, on peut obtenir le lien vers le mp3 ainsi que le site France Inter, de même que le titre et la date de diffusion :

In [8]:
show = all_shows[0]

In [9]:
show.find('a', class_='download-link').attrs['href'].strip()

'http://prevost.pascal.free.fr/public/podcast/sur_les_epaules_de_darwin/Jean-Claude%20Ameisen%20-%20SUR%20LES%20EPAULES%20DE%20DARWIN%2004.09.2010.mp3'

In [10]:
show.find('span').find('a').attrs['href']

'http://www.franceinter.fr/em/sur-les-epaules-de-darwin/94428'

In [11]:
show.find('span').find('a').text

"La théorie de l'évolution de Charles Darwin"

In [12]:
str(show.find('span').find('a').next_sibling)[14:]

'4 Septembre 2010'

Ecrivons maintenant une boucle sur tous les épisodes :

In [13]:
show_data = []
for show in all_shows:
    download_link = show.find('a', class_='download-link')
    description_link = show.find('span').find('a').attrs['href']
    title = show.find('span').find('a').text
    date = str(show.find('span').find('a').next_sibling)[14:]
    if download_link:
        if download_link.attrs['href'].strip().startswith('.'):
            download_link = "http://clementgrimal.fr/darwin" + download_link.attrs['href'].strip()[1:]
        else:
            download_link = download_link.attrs['href'].strip()
        show_data.append([download_link, description_link, title, date])

Construisons un tableau avec ces données :

In [14]:
import pandas as pd

In [15]:
df = pd.DataFrame(show_data, columns=['lien mp3', 'lien description', 'titre', 'date'])
df.head(10)

,lien mp3,lien description,titre,date
0,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,La théorie de l'évolution de Charles Darwin,4 Septembre 2010
1,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Le propre de l'homme?,11 Septembre 2010
2,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Nos mémoires,18 Septembre 2010
3,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Nos émotions,25 Septembre 2010
4,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,"Longévité, jeunesse et vieillissement",2 Octobre 2010
5,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,MORT CELLULAIRE ET SCULPTURE DU VIVANT,9 Octobre 2010
6,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,NAISSANCES,16 Octobre 2010
7,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Biodiversité,23 Octobre 2010
8,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,mort cellulaire et sculpture du vivant (2),30 Octobre 2010
9,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Un voyage avec Oliver Sacks ( 1 ),6 Novembre 2010


In [16]:
df.tail(10)

,lien mp3,lien description,titre,date
371,http://clementgrimal.fr/darwin/files/2017-12-0...,https://www.franceinter.fr/emissions/sur-les-e...,A la recherche des mystères du sommeil (3),9 Décembre 2017
372,http://clementgrimal.fr/darwin/files/2017-12-1...,https://www.franceinter.fr/emissions/sur-les-e...,Explorer,16 Décembre 2017
373,http://clementgrimal.fr/darwin/files/2017-12-2...,https://www.franceinter.fr/emissions/sur-les-e...,Les forges du ciel,23 Décembre 2017
374,http://clementgrimal.fr/darwin/files/2017-12-3...,https://www.franceinter.fr/emissions/sur-les-e...,Les forges du ciel (2),30 Décembre 2017
375,http://clementgrimal.fr/darwin/files/2018-01-0...,https://www.franceinter.fr/emissions/sur-les-e...,La glace et le feu,6 Janvier 2018
376,http://clementgrimal.fr/darwin/files/2018-01-1...,https://www.franceinter.fr/emissions/sur-les-e...,Eclats de passé : le Croissant Fertile,13 Janvier 2018
377,http://clementgrimal.fr/darwin/files/2018-01-2...,https://www.franceinter.fr/emissions/sur-les-e...,Eclats de passé : la révolution Néolithique,20 Janvier 2018
378,http://clementgrimal.fr/darwin/files/2018-01-2...,https://www.franceinter.fr/emissions/sur-les-e...,Eclats de passé : A la recherche des origines ...,27 Janvier 2018
379,http://clementgrimal.fr/darwin/files/2018-02-0...,https://www.franceinter.fr/emissions/sur-les-e...,A la recherche des origines des inégalités (2),3 Février 2018
380,http://clementgrimal.fr/darwin/files/2018-02-1...,https://www.franceinter.fr/emissions/sur-les-e...,A la recherche des origines des inégalités (3),10 Février 2018


In [17]:
df.describe()

,lien mp3,lien description,titre,date
count,381,381,381,381
unique,381,381,310,381
top,http://clementgrimal.fr/darwin/files/2014-11-2...,https://www.franceinter.fr/emissions/sur-les-e...,La glace et le feu,22 Février 2014
freq,1,1,3,1


Nous voilà maintenant en possession des liens vers 381 épisodes publiés sur le site de Clément Grimal. On remarque que certaines d'entre elles sont des rediffusions (310 épisodes uniques sur 381). 

Téléchargeons maintenant les descriptions des émissions.

# Téléchargement des descriptions à partir du site de France Inter 

Nous avons les liens vers les descriptions : il suffit donc de lire la page de chaque émission et d'en extraire le contenu intéressant.

In [18]:
link = df['lien description'][0]
link

'http://www.franceinter.fr/em/sur-les-epaules-de-darwin/94428'

In [19]:
soup = BeautifulSoup(requests.get(link).text, 'html.parser')

In [20]:
article = soup.find('article')
article

<article class="content-body">
<p><strong>Bibliographie:</strong></p>
<p><strong>L'origine des espèces de Charles Darwin aux Editions Flammarion Origines,</strong></p>
<div class="dva_ad inread" id="dva_128139881/RF_france_inter/divers/divers/rg/inread0"></div>
<script type="text/javascript">
    window.dejavuAds = window.dejavuAds || {
        targets: null,
        slots: []
    };
    var deviceSizes = { all: [[2, 2],], desktop: [[533, 300],],  };
    window.dejavuAds.slots.push({
        name: 'dva_128139881/RF_france_inter/divers/divers/rg/inread0',
        id: 'dva_128139881/RF_france_inter/divers/divers/rg/inread0',
        adunit: '128139881/RF_france_inter/divers/divers/rg/inread',
        sizes: deviceSizes,
        targets: { pos: 'inread' }
    });
</script>
<p><strong>Lettres choisies (1828-1859)de Charles Darwin traduit de l'anglais par Mickael Popelard préface de Stefen Jay Gould aux Editions Bayard</strong></p>
<p><strong>L'expression des émotions chez les hommes et les

Il se trouve que des balises script sont contenues dans les épisodes (elles affichent de la publicité sur le site de France Inter).

Si on reformate les tags HTML, on obtient le résultat suivant :

In [23]:
import bs4

In [24]:
def make_description(article, debug=False):
    """Extracts a text-only description of the formatted HTML."""
    desc = ''
    unmatched = []
    for tag in article.children:
        if tag.name == 'p':
            desc += '' + tag.text + '\n'
        elif tag.name == 'h1':
            desc += ' ' + tag.text + '\n'
        elif tag.name == 'h2':
            desc += '  ' + tag.text + '\n'
        elif tag.name == 'h3':
            desc += '   ' + tag.text + '\n'
        elif tag.name == 'blockquote':
            desc += '>' + tag.text + '\n'
        elif tag.name == 'ul':
            for item in [" - " + child.text for child in tag.children if isinstance(child, bs4.Tag)]:
                desc += item
                if not item.endswith('\n'):
                    desc += '\n'
        elif tag.name == 'a':
            desc += tag.attrs['href'] + '\n'
        elif tag.name == 'ol':
            for child in tag.children:
                if isinstance(child, bs4.Tag):
                    desc += child.text + '\n'
        elif tag.name in ['hr', 'aside']:
            desc += make_description(tag, debug=False)
        else:
            unmatched.append(tag.name)
    if debug:
        print(set(unmatched))
    assert set([None, 'div', 'figure']).issuperset(unmatched)
    return desc

Voici donc notre version texte seul pour la description :

In [28]:
desc = make_description(article, debug=False)

print(desc)

Bibliographie:
L'origine des espèces de Charles Darwin aux Editions Flammarion Origines,
Lettres choisies (1828-1859)de Charles Darwin traduit de l'anglais par Mickael Popelard préface de Stefen Jay Gould aux Editions Bayard
L'expression des émotions chez les hommes et les animaux de Charles Darwin aux Editions Rivages
La lettre volée, d'Edgar Allan Poe, Ed Folio Gallimard
La mal-mesure de l'homme de Stephen Jay Gould aux Editions Odile Jacob
Identité et violence: l'illusion du destin d'Amartya Sen aux editions Odile Jacob
"Dans la Lumière et les Ombres. Darwin et le bouleversement du monde" de Jean-Claude Ameisen aux Editions Fayard / Seuil
   invité(s)
   Charles Darwin 1809-1882
   programmation musicale
   Benjamin Biolay
Si tu suis mon regard album: LA SUPERBElabel: NAIVE RECORDSparution: 2010
   Jane Birkin
Apocalypstick album: EX FAN DES SIXTIESlabel: FONTANAparution: 1978
   Luke
LE ROBOT album: LE ROBOTlabel: JIVE EPIC / SONYparution: 2010
   Cat power
THE GREATEST album: THE 

On peut comparer notre version à l'original:

In [29]:
from IPython.display import HTML

In [30]:
HTML(str(article))

On peut maintenant écrire une boucle sur toutes les descriptions d'épisode :

In [31]:
from functools import lru_cache

In [32]:
@lru_cache(maxsize=None)
def link_getter(description_link):
    return requests.get(description_link)

In [33]:
import tqdm

In [34]:
descriptions = []
for description_link in tqdm.tqdm_notebook(df['lien description']):
    soup = BeautifulSoup(link_getter(description_link).text, 'html.parser')
    article = soup.find('article')
    script = article.find('script')
    if script:
        script.extract()
    descriptions.append(make_description(article))

HBox(children=(IntProgress(value=0, max=381), HTML(value='')))

In [35]:
df['description'] = descriptions

In [36]:
df.head(10)

,lien mp3,lien description,titre,date,description
0,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,La théorie de l'évolution de Charles Darwin,4 Septembre 2010,Bibliographie:\nL'origine des espèces de Charl...
1,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Le propre de l'homme?,11 Septembre 2010,programmation musicale\n STAFF BENDA BILI...
2,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Nos mémoires,18 Septembre 2010,programmation musicale\n Iggy POP\nKing o...
3,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Nos émotions,25 Septembre 2010,programmation musicale\n EMILY LOISEAU\nL...
4,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,"Longévité, jeunesse et vieillissement",2 Octobre 2010,programmation musicale\n Bonga\nMona ki g...
5,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,MORT CELLULAIRE ET SCULPTURE DU VIVANT,9 Octobre 2010,programmation musicale\n Nouvelle vague /...
6,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,NAISSANCES,16 Octobre 2010,invité(s)\n René Frydman\nChef du Pôle Mè...
7,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Biodiversité,23 Octobre 2010,"""L'infinité des formes les plus belles et les ..."
8,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,mort cellulaire et sculpture du vivant (2),30 Octobre 2010,« Nous ne nous baignons pas deux fois dans le ...
9,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Un voyage avec Oliver Sacks ( 1 ),6 Novembre 2010,A la rencontre de la singularité de nos mondes...


In [37]:
df.tail(10)

,lien mp3,lien description,titre,date,description
371,http://clementgrimal.fr/darwin/files/2017-12-0...,https://www.franceinter.fr/emissions/sur-les-e...,A la recherche des mystères du sommeil (3),9 Décembre 2017,\n A la recherche des mystères du s...
372,http://clementgrimal.fr/darwin/files/2017-12-1...,https://www.franceinter.fr/emissions/sur-les-e...,Explorer,16 Décembre 2017,\n Explorer\n \nUne heure qu...
373,http://clementgrimal.fr/darwin/files/2017-12-2...,https://www.franceinter.fr/emissions/sur-les-e...,Les forges du ciel,23 Décembre 2017,\n Les forges du ciel\n \nNo...
374,http://clementgrimal.fr/darwin/files/2017-12-3...,https://www.franceinter.fr/emissions/sur-les-e...,Les forges du ciel (2),30 Décembre 2017,\n Les forges du ciel (2)\n ...
375,http://clementgrimal.fr/darwin/files/2018-01-0...,https://www.franceinter.fr/emissions/sur-les-e...,La glace et le feu,6 Janvier 2018,\n La glace et le feu \n \nN...
376,http://clementgrimal.fr/darwin/files/2018-01-1...,https://www.franceinter.fr/emissions/sur-les-e...,Eclats de passé : le Croissant Fertile,13 Janvier 2018,\n Eclats de passé : le Croissant F...
377,http://clementgrimal.fr/darwin/files/2018-01-2...,https://www.franceinter.fr/emissions/sur-les-e...,Eclats de passé : la révolution Néolithique,20 Janvier 2018,\n Eclats de passé : la révolution ...
378,http://clementgrimal.fr/darwin/files/2018-01-2...,https://www.franceinter.fr/emissions/sur-les-e...,Eclats de passé : A la recherche des origines ...,27 Janvier 2018,\n Eclats de passé : A la recherche...
379,http://clementgrimal.fr/darwin/files/2018-02-0...,https://www.franceinter.fr/emissions/sur-les-e...,A la recherche des origines des inégalités (2),3 Février 2018,\n A la recherche des origines des ...
380,http://clementgrimal.fr/darwin/files/2018-02-1...,https://www.franceinter.fr/emissions/sur-les-e...,A la recherche des origines des inégalités (3),10 Février 2018,\n A la recherche des origines des ...


# Taille des fichiers mp3 

Il nous manque encore la récupération des tailles de fichier mp3, qui sera inscrite dans le flux RSS :

In [38]:
byte_lengths = []
for mp3_link in tqdm.tqdm_notebook(df['lien mp3']):
    r = requests.head(mp3_link)
    byte_lengths.append(r.headers['content-length'])

HBox(children=(IntProgress(value=0, max=381), HTML(value='')))

In [39]:
df['byte_length'] = byte_lengths

In [40]:
df.head()

,lien mp3,lien description,titre,date,description,byte_length
0,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,La théorie de l'évolution de Charles Darwin,4 Septembre 2010,Bibliographie:\nL'origine des espèces de Charl...,51120256
1,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Le propre de l'homme?,11 Septembre 2010,programmation musicale\n STAFF BENDA BILI...,52287616
2,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Nos mémoires,18 Septembre 2010,programmation musicale\n Iggy POP\nKing o...,51320960
3,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Nos émotions,25 Septembre 2010,programmation musicale\n EMILY LOISEAU\nL...,54608000
4,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,"Longévité, jeunesse et vieillissement",2 Octobre 2010,programmation musicale\n Bonga\nMona ki g...,52762752


# Ecriture du fichier RSS 

Maintenant que nous avons collecté toutes les émissions, nous pouvons écrire un ficher RSS, comme nous l'avions déjà fait [dans le cas de l'émission Rendez vous avec X](http://flothesof.github.io/podcast-rendez-vous-avec-X-fr.html).

In [41]:
import dateparser

In [42]:
import xml.etree.cElementTree as ET

rss = ET.Element("rss", version="2.0")
channel = ET.SubElement(rss, "channel")
title = ET.SubElement(channel, "title")
title.text = 'Podcast Sur les épaules de Darwin'
description = ET.SubElement(channel, "description")
description.text = "Podcast inofficiel de l'émission Sur les épaules de Darwin, tiré du site http://clementgrimal.fr/darwin/ et https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin"
for index, row in df.iterrows():
    item = ET.SubElement(channel, "item")
    item_title = ET.SubElement(item, "title")
    item_title.text = row['titre']
    item_description = ET.SubElement(item, "description")
    item_description.text = row['description']
    item_pubdate = ET.SubElement(item, "pubDate")
    item_pubdate.text = dateparser.parse(row.date).date().strftime('%a, %d %b %Y 11:00:00')
    item_enclosure = ET.SubElement(item, "enclosure", url='{}'.format(row['lien mp3']),
                                   length=row.byte_length,
                                  type="audio/mpeg")
tree = ET.ElementTree(rss)
tree.write("files/podcast_Sur_les_epaules_de_Darwin.xml", encoding='utf-8')

Le lien vers le fichier généré est ici : **https://raw.githubusercontent.com/flothesof/posts/master/files/podcast_Sur_les_epaules_de_Darwin.xml**.

*Ce billet a été écrit à l'aide d'un notebook Jupyter. Son contenu est sous licence BSD. Une vue statique de ce notebook peut être consultée et téléchargée ici : [20170908_PodcastEpaulesDeDarwin.ipynb](http://nbviewer.ipython.org/urls/raw.github.com/flothesof/posts/master/20170908_PodcastEpaulesDeDarwin.ipynb).*